In [125]:
from keras.applications import inception_v3
from keras import backend as K

K.set_learning_phase(0)


model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

In [126]:
# lower layers produce patterns, higher layers produce more recognizable
# objects

layer_contributions = {
    'mixed2': 0.2,
    'mixed3': .5,
    'mixed4': 50,
    'mixed5': 50,
}

In [127]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])

loss = K.variable(0.)
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output

    # calculate number of elements
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss += coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

In [128]:
dream = model.input

grads = K.gradients(loss, dream)[0]

grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)


def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values


def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print '...Loss value at', i, ':', loss_value
        x += step * grad_values
    return x

### Running Gradient Ascent over different successive scales

In [129]:
# auxillary functions

import scipy
from keras.preprocessing import image

def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
               float(size[0]) / img.shape[1],
               float(size[1]) / img.shape[2],
               1)
    return scipy.ndimage.zoom(img, factors, order=1)

def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    scipy.misc.imsave(fname, pil_img)
    
def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img

def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [130]:
import numpy as np

step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 20

max_loss = 10.  # avoid ugly artifacts

base_image_path = 'east_bay_hills.jpg'
img = preprocess_image(base_image_path)
original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i))
                  for dim in original_shape])
    successive_shapes.append(shape)

# reverse list so in increasing order
successive_shapes = successive_shapes[::-1]


original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print 'Processing image shape', shape
    img = resize_img(img, shape)
    img = gradient_ascent(img,
                          iterations = iterations,
                          step=step,
                          max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img
    
    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale_' + str(shape) + '.png')
    
save_img(img, fname='final_dream.png')

Processing image shape (215, 287)


/home/dhodun/.local/lib/python2.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  app.launch_new_instance()


Processing image shape (302, 402)
Processing image shape (423, 564)


In [131]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "east_bay_hills.jpg")

In [132]:
Image(url= "dream_at_scale_(215, 287).png")

In [133]:
Image(url= "dream_at_scale_(302, 402).png")

In [134]:
Image(url= "dream_at_scale_(423, 564).png")

In [135]:
Image(url= "final_dream.png")